# **Выводы в конце ноутбука:** 

# Fun with language modelling (Запускать на CPU, на GPU не работает( За основу взял Евгения Онегина:) ОЧЕНЬ долго обучалось и слетало, поэтому решил не добавлять к Онегину корпус текста с википедии, точностью получилась не сильно высокой, тк датасет из одного произведения небольшой

Если вы пропустили лекцию, то посмотрите слайды к ней — они где-то есть. Также полезно почитать:

* [Unreasonable effectiveness of RNN](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) (Andrej Karpathy)
* [Официальный пример от PyTorch](https://github.com/pytorch/examples/tree/master/word_language_model)

Рекомендуется заранее всё прочитать, чтобы понять, что от вас хотят. При желании, можете переписать всё так, как подсказывает ваше сердце.

---

In [ ]:
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader

import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## Препроцессинг (3 балла) За основу взял Евгения Онегина:)

Возьмите какие-нибудь сырые данные. Википедия, «Гарри Поттер», «Игра Престолов», тексты Монеточки, твиты Тинькова — что угодно.

Для простоты будем делать char-level модель. Выкиньте из текстов все ненужные символы (можете оставить только алфавит и, пунктуацию). Сопоставьте всем различным символам свой номер. Удобно это хранить просто в питоновском словаре (`char2idx`). Для генерации вам потребуется ещё и обратный словарь (`idx2char`). Вы что-то такое должны были писать на вступительной — можете просто переиспользовать код оттуда.

Заранее зарезервируйте айдишники под служебные символы: `<START>`, `<END>`, `<PAD>`, `<UNK>`.

Клёво будет написать отдельный класс, который делает токенизацию и детокенизацию.

In [ ]:
import string
punctuation1 = list("!\"'(),-.:;?`")
whitespace = list(string.whitespace)
chars = list("абвгдеёжзийклмнопрстуфхцчшщъыьэюя"+"абвгдеёжзийклмнопрстуфхцчшщъыьэюя".upper())
chars0 = chars
s = "А роза упала на лапу Азора!"
cs = list(filter( lambda x: x in punctuation1 + whitespace + chars, s))
# cs

In [ ]:
# !\"'(),-.:;?` - учитываем эти знаки припинания, пробел и буквы

In [ ]:
whitespace

[' ', '\t', '\n', '\r', '\x0b', '\x0c']

In [ ]:
char2idx = {'<START>': 0, 'END': 1, 'PAD': 2, 'UNK': 3, ' ': 4}
for value, index in enumerate(chars+punctuation1):
    char2idx[index] = value+5
print(char2idx)

{'<START>': 0, 'END': 1, 'PAD': 2, 'UNK': 3, ' ': 4, 'а': 5, 'б': 6, 'в': 7, 'г': 8, 'д': 9, 'е': 10, 'ё': 11, 'ж': 12, 'з': 13, 'и': 14, 'й': 15, 'к': 16, 'л': 17, 'м': 18, 'н': 19, 'о': 20, 'п': 21, 'р': 22, 'с': 23, 'т': 24, 'у': 25, 'ф': 26, 'х': 27, 'ц': 28, 'ч': 29, 'ш': 30, 'щ': 31, 'ъ': 32, 'ы': 33, 'ь': 34, 'э': 35, 'ю': 36, 'я': 37, 'А': 38, 'Б': 39, 'В': 40, 'Г': 41, 'Д': 42, 'Е': 43, 'Ё': 44, 'Ж': 45, 'З': 46, 'И': 47, 'Й': 48, 'К': 49, 'Л': 50, 'М': 51, 'Н': 52, 'О': 53, 'П': 54, 'Р': 55, 'С': 56, 'Т': 57, 'У': 58, 'Ф': 59, 'Х': 60, 'Ц': 61, 'Ч': 62, 'Ш': 63, 'Щ': 64, 'Ъ': 65, 'Ы': 66, 'Ь': 67, 'Э': 68, 'Ю': 69, 'Я': 70, '!': 71, '"': 72, "'": 73, '(': 74, ')': 75, ',': 76, '-': 77, '.': 78, ':': 79, ';': 80, '?': 81, '`': 82}


In [ ]:
idx2char = {0: '<START>', 1: 'END', 2: 'PAD', 3: 'UNK', 4: ' '}
for index, value in enumerate(chars+punctuation1):
    idx2char[index+5] = value
print(idx2char)

{0: '<START>', 1: 'END', 2: 'PAD', 3: 'UNK', 4: ' ', 5: 'а', 6: 'б', 7: 'в', 8: 'г', 9: 'д', 10: 'е', 11: 'ё', 12: 'ж', 13: 'з', 14: 'и', 15: 'й', 16: 'к', 17: 'л', 18: 'м', 19: 'н', 20: 'о', 21: 'п', 22: 'р', 23: 'с', 24: 'т', 25: 'у', 26: 'ф', 27: 'х', 28: 'ц', 29: 'ч', 30: 'ш', 31: 'щ', 32: 'ъ', 33: 'ы', 34: 'ь', 35: 'э', 36: 'ю', 37: 'я', 38: 'А', 39: 'Б', 40: 'В', 41: 'Г', 42: 'Д', 43: 'Е', 44: 'Ё', 45: 'Ж', 46: 'З', 47: 'И', 48: 'Й', 49: 'К', 50: 'Л', 51: 'М', 52: 'Н', 53: 'О', 54: 'П', 55: 'Р', 56: 'С', 57: 'Т', 58: 'У', 59: 'Ф', 60: 'Х', 61: 'Ц', 62: 'Ч', 63: 'Ш', 64: 'Щ', 65: 'Ъ', 66: 'Ы', 67: 'Ь', 68: 'Э', 69: 'Ю', 70: 'Я', 71: '!', 72: '"', 73: "'", 74: '(', 75: ')', 76: ',', 77: '-', 78: '.', 79: ':', 80: ';', 81: '?', 82: '`'}


In [ ]:
class Vocab:
    def __init__(self, data):
        
        self.char2idx = {'<START>': 0, '<END>': 1, '<PAD>': 2, '<UNK>': 3, ' ': 4, 'а': 5, 'б': 6, 'в': 7, 'г': 8, 'д': 9, 'е': 10, 'ё': 11, 'ж': 12, 'з': 13, 'и': 14, 'й': 15, 'к': 16, 'л': 17, 'м': 18, 'н': 19, 'о': 20, 'п': 21, 'р': 22, 'с': 23, 'т': 24, 'у': 25, 'ф': 26, 'х': 27, 'ц': 28, 'ч': 29, 'ш': 30, 'щ': 31, 'ъ': 32, 'ы': 33, 'ь': 34, 'э': 35, 'ю': 36, 'я': 37, 'А': 38, 'Б': 39, 'В': 40, 'Г': 41, 'Д': 42, 'Е': 43, 'Ё': 44, 'Ж': 45, 'З': 46, 'И': 47, 'Й': 48, 'К': 49, 'Л': 50, 'М': 51, 'Н': 52, 'О': 53, 'П': 54, 'Р': 55, 'С': 56, 'Т': 57, 'У': 58, 'Ф': 59, 'Х': 60, 'Ц': 61, 'Ч': 62, 'Ш': 63, 'Щ': 64, 'Ъ': 65, 'Ы': 66, 'Ь': 67, 'Э': 68, 'Ю': 69, 'Я': 70, '!': 71, '"': 72, "'": 73, '(': 74, ')': 75, ',': 76, '-': 77, '.': 78, ':': 79, ';': 80, '?': 81, '`': 82, '<SEP>': 83}
        self.idx2char = {0: '<START>', 1: '<END>', 2: '<PAD>', 3: '<UNK>', 4: ' ', 5: 'а', 6: 'б', 7: 'в', 8: 'г', 9: 'д', 10: 'е', 11: 'ё', 12: 'ж', 13: 'з', 14: 'и', 15: 'й', 16: 'к', 17: 'л', 18: 'м', 19: 'н', 20: 'о', 21: 'п', 22: 'р', 23: 'с', 24: 'т', 25: 'у', 26: 'ф', 27: 'х', 28: 'ц', 29: 'ч', 30: 'ш', 31: 'щ', 32: 'ъ', 33: 'ы', 34: 'ь', 35: 'э', 36: 'ю', 37: 'я', 38: 'А', 39: 'Б', 40: 'В', 41: 'Г', 42: 'Д', 43: 'Е', 44: 'Ё', 45: 'Ж', 46: 'З', 47: 'И', 48: 'Й', 49: 'К', 50: 'Л', 51: 'М', 52: 'Н', 53: 'О', 54: 'П', 55: 'Р', 56: 'С', 57: 'Т', 58: 'У', 59: 'Ф', 60: 'Х', 61: 'Ц', 62: 'Ч', 63: 'Ш', 64: 'Щ', 65: 'Ъ', 66: 'Ы', 67: 'Ь', 68: 'Э', 69: 'Ю', 70: 'Я', 71: '!', 72: '"', 73: "'", 74: '(', 75: ')', 76: ',', 77: '-', 78: '.', 79: ':', 80: ';', 81: '?', 82: '`', 83: '<SEP>'}
        self.true_sequence = []

    def true_sequence_get(self):
      return self.true_sequence

    def tokenize(self, sequence):
      # заменить все whitespace на пробелы

        true_sequence = []
        true_sequence = list(filter( lambda x: x in punctuation1 + whitespace + chars0, sequence))
        
        # выполните какой-то базовый препроцессинг
        # например, оставьте только алфавит и пунктуацию

        for i in range(len(true_sequence)):
          if (true_sequence[i] in ['\t', '\n', '\r', '\x0b', '\x0c']):
            true_sequence[i] = ' '

        # true_sequence.append(' ')
        # for i in range(len(true_sequence)-1):
        #   if (true_sequence[i] in ['.', '!', '?']):
        #     if (true_sequence[i+1] == ' '):
        #       true_sequence[i+1] = '<SEP>' # разделение предложений
        #     else:
        #       true_sequence[i] = '<SEP>' # !! нужно добавлять после пунктуации, вставка, увеличивается строка или через append новую строку создать
        self.true_sequence = true_sequence
        return [self.char2idx[char] for char in true_sequence]
    
    def detokenize(self, sequence):
        return ''.join([self.idx2char[idx] for idx in sequence])
    
    def __len__(self):
        return len(self.char2idx)

In [ ]:
s = "А роза упала! На лапу Азора! "

In [ ]:
vocab = Vocab(s)

sample = vocab.tokenize(s)
print(sample)

[38, 4, 22, 20, 13, 5, 4, 25, 21, 5, 17, 5, 71, 4, 52, 5, 4, 17, 5, 21, 25, 4, 38, 13, 20, 22, 5, 71, 4]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/3-Tinkoff/1-DL course/Пушкин_Евгений_Онегин.txt"

In [ ]:
min_len = 40  # предложения с меньшим количеством символов не будут рассматриваться
max_len = 150 # предложения с большим количеством символов будут обрезаться

In [ ]:
text_file = open(data_path, "r")
data = text_file.read()
text_file.close()

In [ ]:
import nltk
import pandas as pd

nltk.download('punkt')
#data = pd.read_csv(filename, encoding= 'unicode_escape')

class TextDataset():
    
    def __init__(self, data_path):
        # загрузите данные
        text_file = open(data_path, "r")
        data = text_file.read()
        text_file.close()

        # обучите вокаб
        self.vocab = Vocab(data)

        # разделите данные на отдельные сэмплы для обучения
        # (просто список из сырых строк)
        # tokenizer = nltk.data.load('tokenizers/punkt/russian.pickle')
        self.data = nltk.tokenize.sent_tokenize(data, language='russian')
        self.data_raw = self.data

        xtrain = []
        ytrain = []
        for elem in self.data:
          lenth = len(elem)
          if lenth > min_len:
            sample, target = self.__get__((elem.ljust(max_len))[:max_len])

            len1 = max_len - len(sample)
            for j in range(len1):
              sample.append(4)

            len1 = max_len - len(target)
            for j in range(len1):
              target.append(4)


            xtrain.append(sample)
            ytrain.append(target)
            # print(sample)

        self.xtrain = xtrain
        self.ytrain = ytrain

        self.__batch__() # готовые xtrain, ytrain по батчам
    
    def __len__(self):
        return len(self.xtrain)
    
    def __get__(self, sample):
        # sample = self.data[idx]
        seq_len = max_len
        sample = self.vocab.tokenize(sample)
        target = sample[1:max_len]
        target.append(4) # знак пробела
        # уже предобработано до max_lenth sample = # паддинг до maxlen (см. дальше)
        return sample, target

    def __batch__(self):

      totalx = []
      totaly = []
      batches = self.__len__()//batch_size
      for i in range(batches):
        sample = self.xtrain[i*batch_size:(i+1)*batch_size]
        target = self.ytrain[i*batch_size:(i+1)*batch_size]
        #print(sample)
        #print()
        #print(target)
        #sample = torch.as_tensor(np.array(sample)).zero_().view(-1, batch_size)
        #target = torch.as_tensor(np.array(target)).zero_().view(-1, batch_size)
        # sample = torch.LongTensor(seq_len * batch_size).zero_().view(-1, batch_size).to(device)

        totalx.append(sample)
        totaly.append(target)

      # print(totaly)
      # print(totaly[0])
      # print(totaly[0][0])

      train_size = int(0.9 * len(dataset))
      # test_size = len(dataset) - train_size

      self.data = torch.as_tensor(totalx[:train_size])#.zero_().view(-1, batches).to(device)
      self.targets = torch.as_tensor(totaly[:train_size])#.zero_().view(-1, batches).to(device)

      self.data_test = torch.as_tensor(totalx[train_size:])#.zero_().view(-1, batches).to(device)
      self.targets_test = torch.as_tensor(totaly[train_size:])#.zero_().view(-1, batches).to(device)

      


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Если у вас какой-то большой массив текста (скажем, статьи Википедии), вы можете просто нарезать из него кусочки фиксированной длины и так их подавать в модель.

Если же вы хотите приключений, то можно разбить этот текст на предложения (`nltk.sent_tokenize`), и тогда все примеры будут разной длины. По соображениям производительности, вы не хотите использовать самые длинные и самые короткие сэмплы, поэтому имеет смысл обрезать их по длине.

Разобьём на обучение и валидацию:

In [ ]:
batch_size = 16
dataset = TextDataset(data_path)

train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, test_size])

In [ ]:
# train_set.dataset

In [ ]:
# dataset.data

In [ ]:
# dataset.targets

In [ ]:
plt.hist([len(x) for x in dataset.data_raw], bins=350)
plt.title('Распределение числа слов в предложениях.')
plt.xlabel('Число слов')
plt.xlim((0, 300))
# plt.show()

## Модель (3 балла)

Примерно такое должно зайти:

* Эмбеддинг
* LSTM / GRU
* Линейный слой
* Softmax

In [ ]:
class LM(nn.Module):
    def __init__(self, vocab_size, ntoken, embedding_dim, hidden_dim, num_layers, dropout, tie_weights):
        super().__init__()

        rnn_type = "LSTM"
        self.rnn_type = rnn_type
        self.nhid = hidden_dim
        self.nlayers = num_layers
        ntoken = vocab_size

        self.encoder = nn.Embedding(ntoken, embedding_dim)
        self.rnn = nn.GRU(embedding_dim, hidden_dim, num_layers, dropout=dropout)
        self.decoder = nn.Linear(self.nhid, ntoken)
        self.drop = nn.Dropout(dropout)
        
        if tie_weights:
            # "Using the Output Embedding to Improve Language Models" (Press & Wolf 2016)
            # https://arxiv.org/abs/1608.05859
            assert hidden_dim == embedding_dim
            self.decoder.weight = self.encoder.weight

    def forward(self, input, hidden):
        emb = self.drop(self.encoder(input))
        output, hidden = self.rnn(emb, hidden)
        output = self.drop(output)
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded.view(output.size(0), output.size(1), decoded.size(1)), hidden

    def init_hidden(self, batch_size):
        # начальный хидден должен быть нулевой
        # (либо хоть какой-то константный для всего обучения)
        return torch.zeros(batch_size, self.nhid)

## Обучение

In [ ]:
def get_accuracy(logit, target, batch_size):
    ''' Obtain accuracy for training round '''
    corrects = (torch.max(logit, 1)[1].view(target.size()).data == target.data).sum()
    accuracy = 100.0 * corrects/batch_size
    return accuracy.item()

In [ ]:
epochs = 5
lr = 1e-3 
batch_size = 16

device = torch.device('cpu')

model = LM(
    vocab_size = len(dataset.vocab),
    ntoken = max_len,
    embedding_dim = 128,
    hidden_dim = 128,
    num_layers = 1,
    dropout = 0.1,
    tie_weights=True
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()

trainloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test = DataLoader(test_set, batch_size=batch_size)

In [ ]:
train_len = len(dataset.data)
test_len = len(dataset.data_test)

for e in range(epochs):
    break
    train_running_loss = 0.0
    train_acc = 0.0
    model.train()

    for i in range(train_len):
        model.zero_grad()
        optimizer.zero_grad()
        
        # 0. Распакуйте данные на нужное устройство
        inputs = dataset.data[i]
        labels = dataset.targets[i]
        # inputs = inputs.view(-1, 28,28) 
        # 1. Инициилизируйте hidden
        hidden = model.init_hidden(max_len) 
        # 2. Прогоните данные через модель, получите предсказания на каждом токене
        output, hidden = model(data, hidden)
        # 3. Посчитайте лосс (maxlen независимых классификаций) и сделайте backward()

        loss = criterion(output, labels)
        loss.backward()

        # 4. Клипните градиенты -- у RNN-ок с этим часто бывают проблемы
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.2)
        # 5. Залоггируйте лосс куда-нибудь
        
        optimizer.step()
    
        train_running_loss += loss.detach().item()
        train_acc += get_accuracy(labels, labels, batch_size)
         
    
    for i in range(test_len):
        model.eval()

        print('Epoch:  %d | Loss: %.4f | Train Accuracy: %.2f' 
          %(e, train_running_loss / i, train_acc/i))
        
        # здесь нужно сделать то же самое, только без backward

In [ ]:
# Возникла ошибка "TypeError: embedding(): argument 'indices' (position 2) must be Tensor, not str"

Ниже другая реализация обучения

In [ ]:
# dataset = TextDataset(data_path)
# utf-8 !!
text_file = open(data_path, "r")
data = text_file.read()
text_file.close()

# обучите вокаб
vocab1 = Vocab(data)
vocab1.tokenize(data)

txt_data = vocab1.true_sequence_get() # input data
# txt_data = open('input.txt', 'r').read() # test external files

chars = list(set(txt_data)) # split and remove duplicate characters. convert to list.

num_chars = len(chars) # the number of unique characters
txt_data_size = len(txt_data)

print("unique characters : ", num_chars) # You can see the number of unique characters in your input data.
print("txt_data_size : ", txt_data_size)

unique characters :  70
txt_data_size :  169541


In [ ]:
# one hot encode
char_to_int = dict((c, i) for i, c in enumerate(chars)) # "enumerate" retruns index and value. Convert it to dictionary
int_to_char = dict((i, c) for i, c in enumerate(chars))
print(char_to_int)
print("----------------------------------------------------")
print(int_to_char)
print("----------------------------------------------------")
# integer encode input data
integer_encoded = [char_to_int[i] for i in txt_data] # "integer_encoded" is a list which has a sequence converted from an original data to integers.
print(integer_encoded)
print("----------------------------------------------------")
print("data length : ", len(integer_encoded))

{'П': 0, 'т': 1, 'Г': 2, 'Р': 3, 'ь': 4, ';': 5, 'о': 6, 'Е': 7, 'ъ': 8, 'л': 9, 'Б': 10, 'м': 11, 'Э': 12, 'в': 13, 'у': 14, 'ц': 15, 'И': 16, ',': 17, 'р': 18, 'к': 19, 'д': 20, 'Ч': 21, 'ч': 22, 'я': 23, 'Я': 24, 'Х': 25, '(': 26, 'Д': 27, 'э': 28, 'М': 29, 'З': 30, 'ф': 31, 'В': 32, 'Ф': 33, 'х': 34, 'щ': 35, 'Ю': 36, '-': 37, 'з': 38, 'С': 39, 'г': 40, 'ю': 41, 'е': 42, 'ш': 43, ':': 44, 'Л': 45, ' ': 46, 'У': 47, 'ё': 48, 'Ц': 49, 'й': 50, ')': 51, 'с': 52, '.': 53, '!': 54, 'п': 55, 'н': 56, 'б': 57, 'О': 58, 'К': 59, 'Н': 60, '?': 61, 'а': 62, 'Т': 63, 'Ш': 64, 'ж': 65, 'и': 66, 'Ж': 67, 'А': 68, 'ы': 69}
----------------------------------------------------
{0: 'П', 1: 'т', 2: 'Г', 3: 'Р', 4: 'ь', 5: ';', 6: 'о', 7: 'Е', 8: 'ъ', 9: 'л', 10: 'Б', 11: 'м', 12: 'Э', 13: 'в', 14: 'у', 15: 'ц', 16: 'И', 17: ',', 18: 'р', 19: 'к', 20: 'д', 21: 'Ч', 22: 'ч', 23: 'я', 24: 'Я', 25: 'Х', 26: '(', 27: 'Д', 28: 'э', 29: 'М', 30: 'З', 31: 'ф', 32: 'В', 33: 'Ф', 34: 'х', 35: 'щ', 36: 'Ю', 37

In [ ]:
# hyperparameters

iteration = 300
sequence_length = 150
batch_size = round((txt_data_size /sequence_length)+0.5) # = math.ceil
hidden_size = 128  # size of hidden layer of neurons.  
learning_rate = 1e-1


# model parameters

W_xh = np.random.randn(hidden_size, num_chars)*0.01     # weight input -> hidden. 
W_hh = np.random.randn(hidden_size, hidden_size)*0.01   # weight hidden -> hidden
W_hy = np.random.randn(num_chars, hidden_size)*0.01     # weight hidden -> output

b_h = np.zeros((hidden_size, 1)) # hidden bias
b_y = np.zeros((num_chars, 1)) # output bias

h_prev = np.zeros((hidden_size,1)) # h_(t-1)

In [ ]:
def forwardprop(inputs, targets, h_prev):
        
    # Since the RNN receives the sequence, the weights are not updated during one sequence.
    xs, hs, ys, ps = {}, {}, {}, {} # dictionary
    hs[-1] = np.copy(h_prev) # Copy previous hidden state vector to -1 key value.
    loss = 0 # loss initialization
    
    for t in range(len(inputs)): # t is a "time step" and is used as a key(dic).  
        
        xs[t] = np.zeros((num_chars,1)) 
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(W_xh, xs[t]) + np.dot(W_hh, hs[t-1]) + b_h) # hidden state. 
        ys[t] = np.dot(W_hy, hs[t]) + b_y # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars. 
        # Softmax. -> The sum of probabilities is 1 even without the exp() function, but all of the elements are positive through the exp() function.
 
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss). Efficient and simple code

#         y_class = np.zeros((num_chars, 1)) 
#         y_class[targets[t]] =1
#         loss += np.sum(y_class*(-np.log(ps[t]))) # softmax (cross-entropy loss)        

    return loss, ps, hs, xs 

def backprop(ps, inputs, hs, xs):

    dWxh, dWhh, dWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy) # make all zero matrices.
    dbh, dby = np.zeros_like(b_h), np.zeros_like(b_y)
    dhnext = np.zeros_like(hs[0]) # (hidden_size,1) 

    # reversed
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t]) # shape (num_chars,1).  "dy" means "dloss/dy"
        dy[targets[t]] -= 1 # backprop into y. After taking the soft max in the input vector, subtract 1 from the value of the element corresponding to the correct label.
        dWhy += np.dot(dy, hs[t].T)
        dby += dy 
        dh = np.dot(W_hy.T, dy) + dhnext # backprop into h. 
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity #tanh'(x) = 1-tanh^2(x)
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(W_hh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]: 
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients.  
    
    return dWxh, dWhh, dWhy, dbh, dby

In [ ]:
data_pointer = 0

# memory variables for Adagrad
mWxh, mWhh, mWhy = np.zeros_like(W_xh), np.zeros_like(W_hh), np.zeros_like(W_hy)
mbh, mby = np.zeros_like(b_h), np.zeros_like(b_y) 

In [ ]:
learning_rate = 1e-1
iteration = 50

for i in range(iteration):
    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    for b in range(batch_size):
        
        inputs = [char_to_int[ch] for ch in txt_data[data_pointer:data_pointer+sequence_length]]
        targets = [char_to_int[ch] for ch in txt_data[data_pointer+1:data_pointer+sequence_length+1]] # t+1        
            
        if (data_pointer+sequence_length+1 >= len(txt_data) and b == batch_size-1): # processing of the last part of the input data. 
#             targets.append(char_to_int[txt_data[0]])   # When the data doesn't fit, add the first char to the back.
            targets.append(char_to_int[" "])   # When the data doesn't fit, add space(" ") to the back.


        # forward
        loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
#         print(loss)
    
        # backward
        dWxh, dWhh, dWhy, dbh, dby = backprop(ps, inputs, hs, xs) 
        
        
    # perform parameter update with Adagrad
        for param, dparam, mem in zip([W_xh, W_hh, W_hy, b_h, b_y], 
                                    [dWxh, dWhh, dWhy, dbh, dby], 
                                    [mWxh, mWhh, mWhy, mbh, mby]):
            mem += dparam * dparam # elementwise
            param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update      
    
        data_pointer += sequence_length # move data pointer

    print ('iter %d, loss: %f' % (i, loss/100)) # print progress    
    # if i % 100 == 0:
    #     print ('iter %d, loss: %f' % (i, loss)) # print progress

iter 0, loss: 1.233516
iter 1, loss: 1.208459
iter 2, loss: 1.206680
iter 3, loss: 1.201401
iter 4, loss: 1.204725
iter 5, loss: 1.186245
iter 6, loss: 1.191167
iter 7, loss: 1.179950
iter 8, loss: 1.170449
iter 9, loss: 1.161095
iter 10, loss: 1.159849
iter 11, loss: 1.158110
iter 12, loss: 1.155956
iter 13, loss: 1.160885
iter 14, loss: 1.156991
iter 15, loss: 1.158801
iter 16, loss: 1.162412
iter 17, loss: 1.158107
iter 18, loss: 1.149162
iter 19, loss: 1.146293
iter 20, loss: 1.142838
iter 21, loss: 1.140843
iter 22, loss: 1.140057
iter 23, loss: 1.140693
iter 24, loss: 1.142841
iter 25, loss: 1.143522
iter 26, loss: 1.143999
iter 27, loss: 1.144792
iter 28, loss: 1.145175
iter 29, loss: 1.145493
iter 30, loss: 1.145491
iter 31, loss: 1.145592
iter 32, loss: 1.145357
iter 33, loss: 1.144153
iter 34, loss: 1.140239
iter 35, loss: 1.135426
iter 36, loss: 1.133199
iter 37, loss: 1.132159
iter 38, loss: 1.131303
iter 39, loss: 1.130745
iter 40, loss: 1.130380
iter 41, loss: 1.130079
it

## Спеллчекер (3 балла)

Из языковой модели можно сделать простенький спеллчекер: можно визуализировать лоссы на каждом символе (либо какой-нибудь другой показатель неуверенности).

Бонус: можете усреднить перплексии по словам и выделять их, а не отдельные символы.

In [ ]:
from IPython.core.display import display, HTML

def print_colored(sequence, intensities, delimeter=''):
    html = delimeter.join([
        f'<span style="background: rgb({255}, {255-x}, {255-x})">{c}</span>'
        for c, x in zip(sequence, intensities) 
    ])
    display(HTML(html))

print_colored('Налейте мне экспрессо'.split(), [0, 0, 100], ' ')

sequence = 'Эту домашку нужно сдать втечении двух недель'
intensities = [0]*len(sequence)
intensities[25] = 50
intensities[26] = 60
intensities[27] = 70
intensities[31] = 150
print_colored(sequence, intensities)

In [ ]:
def spellcheck(sequence):
    # векторизуйте sequence; паддинги делать не нужно
    # sequence = ... сделано ниже

    h_prev = np.zeros((hidden_size,1)) # reset RNN memory
    data_pointer = 0 # go from start of data
    
    inputs = [char_to_int[ch] for ch in sequence]
    targets = [char_to_int[ch] for ch in sequence[1:]]
    targets.append(char_to_int[" "])    

    # forward
    loss, ps, hs, xs = forwardprop(inputs, targets, h_prev)
    
    # прогоните модель и посчитайте лосс, но не усредняйте
    # с losses можно что-нибудь сделать для визуализации; например, в какую-нибудь степень возвести
    spellcheck_losses = loss # 2^loss
    # print(spellcheck_losses)
    
    print_colored(sequence, [spellcheck_losses]*len(sequence))

In [ ]:
sequences = ['В этом претложениии очен много очепяток.', 
             'Здесь появилась лишнняя буква.', 
             'В этом предложении все нормально.', 
             'Чтонибудь пишеться чериз дефис.', 
             'Слова нрпдзх не сущесдвует.']

for sequence in sequences:
    spellcheck(sequence)

## Генерация предложений (3 балла)

* Поддерживайте hidden state при генерации. Не пересчитывайте ничего больше одного раза.
* Прикрутите температуру: это когда при сэмплировании все логиты (то, что перед софтмаксом) делятся на какое-то число (по умолчанию 1, тогда ничего не меняется). Температура позволяет делать trade-off между разнообразием и правдоподобием (подробнее — см. блог Карпатого).
* Ваша реализация должна уметь принимать строку seed — то, с чего должно начинаться сгенерированная строка.

In [ ]:
def sample(num_tokens, seed="",  temperature=1.0):
    input = np.zeros((num_chars, 1))
    # векторизауем seed, чтобы на первом этап получить нужный hidden  
    for test_char in seed[0]: 
      input[char_to_int[test_char]] = 1
    ixes = []
    h = np.zeros((hidden_size,1))

    continuation = ""

    for t in range(num_tokens):
        h = np.tanh(np.dot(W_xh, input) + np.dot(W_hh, h) + b_h) 
        y = np.dot(W_hy, h) + b_y
        #print(y)
        # y = y/temperature # !! temperature | test later
        p = np.exp(y) / np.sum(np.exp(y)) 
        ix = np.random.choice(range(num_chars), p=p.ravel()) # ravel -> rank0
        # "ix" is a list of indexes selected according to the soft max probability.
        input = np.zeros((num_chars, 1)) # init
        input[ix] = 1 
        ixes.append(ix) # list
    continuation = ''.join(int_to_char[i] for i in ixes)
    return continuation

In [ ]:
beginnings = ['Шел медведь по лесу', 
              'Встретились англичанин, американец и русский. Англичанин говорит:',
              'Так вот, однажды качки решили делать ремонт',
              'Поручик Ржевский был',
              'Идёт Будда с учениками по дороге',
              'Мюллер: Штирлиц, где вы были в 1938 году?',
              'Засылают к нам американцы шпиона под видом студента',
              'Подъезжает электричка к Долгопе:']

for beginning in beginnings:
    print(f'{beginning}... {sample(10, beginning, temperature=1.0)}')
    print()

Шел медведь по лесу...  видят он 

Встретились англичанин, американец и русский. Англичанин говорит:...  друг мой 

Так вот, однажды качки решили делать ремонт...  в поместь

Поручик Ржевский был...  пьян и бы

Идёт Будда с учениками по дороге... , видеть г

Мюллер: Штирлиц, где вы были в 1938 году?...  Помесье в

Засылают к нам американцы шпиона под видом студента...  нам друг 

Подъезжает электричка к Долгопе:...  Напоре в 



# Выводы:

За основу взял Евгения Онегина:)

1. Реализовал препроцессинг (загрузка датасета, очистка, формирование dataset с помощью классов **Vocab**(), **TextDataset**()
2. Обучил **LM**
3. Реализовал **SpeelChecker** и **выделил** цветом, чем больше ошибок, тем сильнее подсветка цветом
4. Сгенерировал предложения с **температурой**, на вход - seed, на выходе - сгенерированная строка